In [1]:
pip install qiskit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install qiskit qiskit-aer


Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip show qiskit qiskit-aer


Name: qiskit
Version: 1.3.0
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: /home/ep/.local/lib/python3.10/site-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-aer
---
Name: qiskit-aer
Version: 0.15.1
Summary: Aer - High performance simulators for Qiskit
Home-page: https://github.com/Qiskit/qiskit-aer
Author: AER Development Team
Author-email: qiskit@us.ibm.com
License: Apache 2.0
Location: /home/ep/.local/lib/python3.10/site-packages
Requires: numpy, psutil, qiskit, scipy
Required-by: 


In [4]:
from qiskit_aer import AerSimulator
print("AerSimulator is installed successfully.")


AerSimulator is installed successfully.


In [5]:
# Quantum Support Vector Machine (QSVM) with Iris Dataset

import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC

# Load the Iris dataset
data = load_iris()
X = data.data[:100, :2]  # Use only the first two features and first two classes
y = data.target[:100]

# Encode labels to 0 and 1
le = LabelEncoder()
y = le.fit_transform(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Quantum kernel definition
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

def quantum_kernel(x1, x2):
    """Quantum Kernel using a simple feature map."""
    @qml.qnode(dev)
    def kernel_circuit(x1, x2):
        # Encode the first vector
        for i in range(n_qubits):
            qml.RY(x1[i], wires=i)
        
        # Encode the second vector
        for i in range(n_qubits):
            qml.RY(-x2[i], wires=i)

        # Apply an entangling layer
        for i in range(n_qubits - 1):
            qml.CZ(wires=[i, i + 1])

        return qml.probs(wires=[0])

    return kernel_circuit(x1, x2)[0]

# Precompute the kernel matrix
X_train_q = [qml.math.cast(x, float) for x in X_train]
X_test_q = [qml.math.cast(x, float) for x in X_test]

n_train = len(X_train)
k_train = np.zeros((n_train, n_train))

for i in range(n_train):
    for j in range(n_train):
        k_train[i, j] = quantum_kernel(X_train_q[i], X_train_q[j])

# Train the classical SVM with quantum kernel
svm = SVC(kernel='precomputed')
svm.fit(k_train, y_train)

# Compute the kernel matrix for test data
n_test = len(X_test)
k_test = np.zeros((n_test, n_train))

for i in range(n_test):
    for j in range(n_train):
        k_test[i, j] = quantum_kernel(X_test_q[i], X_train_q[j])

# Evaluate the model
accuracy = svm.score(k_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")


ModuleNotFoundError: No module named 'pennylane'